In [61]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
data = pd.read_csv('train.csv')

In [12]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [11]:
data.shape

(891, 12)

In [14]:
data.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [23]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [24]:
# what to do with objects? aka NaNs - better do smth
# what to do with dates?
def show_basic_plots(df=data, target_var=None, vars_subset=None, use_sublot=True):
    # Presets:
    if vars_subset is None:
        vars_subset = df.columns.tolist()
        
    if target_var is None:
        target_var = df.columns[0]
    
    if target_var in vars_subset:
        vars_subset.remove(target_var)
    
    colors_in_use = ['#2C3E50', '#537EA2', '#858F84', '#42A593',
                     '#873E23', '#CFD1A1', '#6A744F', '#BDBDC5',
                     '#7EA253', '#EDB676', '#C26D40']
    
    dtypes_num = ['int64', 'int32', 'int16', 'float64', 'float32', 'float16']
    dtypes_str = ['object', 'category']
    vars_num = df.loc[:, vars_subset].select_dtypes(include=dtypes_num).columns
    vars_str = df.loc[:, vars_subset].select_dtypes(include=dtypes_str).columns
    
    # For categorical variables:
    fig_str_lst = [None]*len(vars_str)
    for i in range(0, len(vars_str)):
        fig = go.Figure()
        fig.add_trace(go.Histogram(x=df.loc[:, vars_str[i]],
                                   name=vars_str[i],
                                   showlegend=True))
        fig.update_traces(marker_color=colors_in_use[1],
                          marker_line_color='rgb(8,48,107)',
                          marker_line_width=1.5,
                          opacity=0.8)
        fig.update_layout(xaxis_type='category',
                          xaxis_title=vars_str[i],
                          paper_bgcolor='#F5F5F5',
                          plot_bgcolor='#F5F5F5')
        fig.update_yaxes(gridcolor='#DCDCDC')
        fig.update_xaxes(linecolor='#DCDCDC')
        fig_str_lst[i] = fig
        
    # For categorical variables (2+ vars):
    fig_str_lst_mix = [None]*(len(vars_str)**2)
    for i in range(0, len(vars_str)):
        for j in range(0, len(vars_str)):
            fig = go.Figure()
            fig.add_trace(go.Histogram(x=df.loc[:, vars_str[i]],
                                       y=df.loc[:, vars_str[j]],
                                       showlegend=True))
            fig_str_lst_mix[i+j] = fig
    # For categorical versus target:
    
    
    # For numerical - density plots, box plots, violin plots;
    
    # For numerical 2+ - scatter+lm; 
    # for all - density plots, box plots, violin plots, bar plots
    # for all 2+ - scatter+lm by color
    return([fig_str_lst, fig_str_lst_mix])


#fig.update_layout(barmode='stack')
#fig.update_xaxes(categoryorder='category ascending')
#fig.show()

In [57]:
?go.Bar

In [68]:
dtypes_str = ['object', 'category']
vars_str = data.select_dtypes(include=dtypes_str).columns
#fig = go.Figure()
i = 1
j = 4
#fig.add_trace(go.Histogram(x=data.loc[:, vars_str[i]], showlegend=True))
#fig.add_trace(go.Histogram(x=data.loc[:, vars_str[j]], showlegend=True))
#fig
colors_in_use = ['#2C3E50', '#537EA2', '#858F84', '#42A593',
                     '#873E23', '#CFD1A1', '#6A744F', '#BDBDC5',
                     '#7EA253', '#EDB676', '#C26D40']
fig = px.histogram(data, x=vars_str[i], color=vars_str[j], color_discrete_sequence=colors_in_use[0:1])
fig.update_traces(marker_color=colors_in_use[0],
                          marker_line_color='rgb(8,48,107)',
                          marker_line_width=1.5,
                          opacity=0.8)
fig.update_layout(xaxis_type='category',
                          xaxis_title=vars_str[i],
                          paper_bgcolor='#F5F5F5',
                          plot_bgcolor='#F5F5F5')
fig.update_yaxes(gridcolor='#DCDCDC')
fig.update_xaxes(linecolor='#DCDCDC')
fig.show()

In [43]:
tmp = show_basic_plots(data)

len(tmp[1])

tmp[0][3].show()

In [230]:
vars_subset = data.columns.tolist()
target_var = data.columns[0]
if target_var in vars_subset:
    vars_subset.remove(target_var)

['Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [227]:
type(vars_subset)

method

In [169]:
df = data
i = 1
dtypes_str = ['object', 'category']
vars_str = df.select_dtypes(include=dtypes_str).columns

fig = go.Figure()
fig.add_trace(go.Histogram(x=df.loc[:, ],
                           name=vars_str[i]))
fig.update_layout(xaxis_type='category', xaxis_title=vars_str[i])
fig.show()

In [155]:
i = vars_str[1]
data[data.columns[data.columns.isin([i])]]



,Sex
0,male
1,female
2,female
3,female
4,male
...,...
886,male
887,female
888,female
889,male


In [146]:
for i in range(0, len(vars_str)):
    #print(data.iloc[:, i])
    print(vars_str[i])

Name
Sex
Ticket
Cabin
Embarked
